In [1]:
import pandas as pd
pd.options.plotting.backend = "plotly"

Starting with some datasheet parameters of a BYD battery for residential use cases with 5.12 kWh energy and 5.1 kW power:

In [ ]:
datasheet = {
    "brand": "BYD",
    "model": "HVS 5.1",
    "width": 0.585,
    "height": 0.712,
    "depth": 0.298,
    "weight": 91,
    "chemistry": "LFPGraphite",
    "modules": 2,
    "energy_wh": 5120,
    "voltage": 204,
    "max_power_w": 5100,
}

And given a realistic dispatch series of, again, a residential use case with 2.7 kWp and about 6 MWh yearly generation:

In [40]:
import pickle

dispatch = pickle.load(open("dispatch.pkl", "rb")).loc["2021":"2040"] / 1000

We can create a default LFP battery model and adjust it with the datasheet parameters:

In [3]:
from pandas import DataFrame
import PySAM.BatteryStateful as bt
from PySAM.BatteryTools import battery_model_sizing

b = bt.default(datasheet["chemistry"])

battery_model_sizing(
    model=b,
    desired_power=datasheet["max_power_w"] / 1000,
    desired_capacity=datasheet["energy_wh"] / 1000,
    desired_voltage=datasheet["voltage"],
)

b.Controls.control_mode = 1
b.Controls.dt_hr = 1
b.ParamsCell.minimum_SOC = 5
b.ParamsCell.maximum_SOC = 95
b.ParamsCell.initial_SOC = 50
b.Controls.input_power = 0
b.setup()

results = []
for i in dispatch:
    b.Controls.input_power = i
    b.execute(0)
    results.append({"Power": b.StatePack.P, "SOC": b.StatePack.SOC})

df = DataFrame(results, index=dispatch.index)
df["Battery to load"] = df[df["Power"] > 0]["Power"]
df["System to battery"] = -df[df["Power"] < 0]["Power"]
df["Battery to load"] = df["Battery to load"].fillna(0.0)
df["System to battery"] = df["System to battery"].fillna(0.0)

The power flow is typical for a self-consumption use case, where the battery charges during daylight (when the system is producing more energy than required by the load) and fully discharges during night:

In [42]:
df[["System to battery", "Battery to load"]].groupby(df.index.hour).mean().plot.bar(title="Average hourly power flow")

While the power flow looks fine, the lifetime losses look like weird though. The battery-to-load flow:

- Decreases about 2 % during ~3 years
- Then it decreases about 1 % during 2 more years
- Then it kind of flattens, reaching 88 % after 20 years

In [44]:
aux = df["Battery to load"].groupby(df.index.year).mean()
aux = aux / aux.iloc[0] * 100
aux.plot.bar(title="Normalized yearly average battery-to-load power flow")